In [1]:
import pandas as pd

# Loading Memorization Evals
> Memorization evals are dataframes (1 per model checkpoint)

> Each dataframe has two columnns: index of data and memorization score of data (accuracy between true continuation and model generation)

In [3]:
models = ['13b', '6.7b', '2.7b', '1.3b', '800m', '350m', '125m', '19m']
checkpoints = [23000, 43000, 63000, 83000, 103000, 123000, 143000]
checkpoint_names = ['23m', '44m', '65m', '85m', '105m', '126m', '146m']
filepath = '/fsx/orz/memorization-evals'

In [4]:
import os
import pandas as pd
from tqdm.auto import tqdm

tqdm.pandas()

In [ ]:
memorization_results = {}
for model in models:
    
    for checkpoint in tqdm(checkpoints, desc=model):
        filename = os.path.join(filepath, f'memorization_{model}_{checkpoint}.hdf')
        model_name = f'{model}-{checkpoint}'
        try:
            memorization_results[model_name] = pd.read_hdf(filename, key='memorization')
        except Exception as e:
            csv = pd.read_csv(os.path.join(filepath, f'memorization_results_{model}_{checkpoint}.csv'))
            csv.to_hdf(filename, key='memorization', index=False)
            memorization_results[model_name] = csv

13b:   0%|          | 0/7 [00:00<?, ?it/s]

6.7b:   0%|          | 0/7 [00:00<?, ?it/s]

2.7b:   0%|          | 0/7 [00:00<?, ?it/s]

1.3b:   0%|          | 0/7 [00:00<?, ?it/s]

800m:   0%|          | 0/7 [00:00<?, ?it/s]

350m:   0%|          | 0/7 [00:00<?, ?it/s]

125m:   0%|          | 0/7 [00:00<?, ?it/s]

# Loading Megatron Dataset

In [11]:
from megatron.data.data_utils import build_the_dataset

fatal: not a git repository (or any parent up to mount point /)
Stopping at filesystem boundary (GIT_DISCOVERY_ACROSS_FILESYSTEM not set).


In [13]:
dataset = build_the_dataset(
        data_prefix = '/fsx/pile/pile_20B_tokenizer_text_document', # Replace with the path of pile document
        name = 'train_0',
        data_impl='mmap',
        num_samples=131727360,
        seq_length=2048,
        seed=1234,
        skip_warmup=True,
    )

    reading sizes...
    reading pointers...
    reading document index...
    creating numpy buffer of mmap...
    creating memory view of numpy buffer...
    train_0:
     no. of documents:210604984


Memorization evals are in Pandas DataFrames. We can use Pandarallel, if required to speed up .apply operations

In [29]:
!pip3 install pandarallel

     |████████████████████████████████| 110 kB 29.8 MB/s eta 0:00:01
  Created wheel for pandarallel: filename=pandarallel-1.6.3-py3-none-any.whl size=16449 sha256=420d869fee4d17f949725f8fd5b680063135fd0601c69f976f57c811e13e68e0
  Stored in directory: /admin/home-orz/.cache/pip/wheels/16/f5/91/d7efa7c4911ae1cf1aff825f902382eb69a1855f9987a7d17c
Successfully built pandarallel


In [32]:
from pandarallel import pandarallel
pandarallel.initialize(progress_bar=True)

INFO: Pandarallel will run on 48 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.
